<a href="https://colab.research.google.com/github/akshaygrao77/DeepLearning-Assignment2/blob/main/PART_A.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%capture
!curl -SL https://storage.googleapis.com/wandb_datasets/nature_12K.zip > dataset.zip

In [2]:
!unzip dataset.zip

Archive:  dataset.zip
replace inaturalist_12K/.DS_Store? [y]es, [n]o, [A]ll, [N]one, [r]ename: N


In [3]:
#Import and install required libraries
import os
import glob
import tensorflow as tf

import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential, Model
from keras.layers import *
from keras.optimizers import *
from keras.preprocessing.image import ImageDataGenerator
from keras.utils.vis_utils import plot_model
from keras.callbacks import History
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import os
import cv2
!pip install wandb
!wandb login
import wandb
from wandb.keras import WandbCallback
import os
from PIL import Image

%matplotlib inline
%config InlineBackend.figure_format = 'svg'
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from keras.layers import LeakyReLU
from keras.layers import PReLU
from keras.initializers import Constant

wandb: Currently logged in as: akshaygrao (use `wandb login --relogin` to force relogin)


In [4]:
import argparse

In [5]:
train_Data_Path = os.getcwd()+"/inaturalist_12K/train"
test_Data_Path = os.getcwd()+"/inaturalist_12K/val"

width, height = 200, 200

batch_size = 128

In [6]:
wandb.init(project="DLAssignment2", entity='cs21s002-ee21s113-dlassignment-1')
#wandb.init(project="DeepLearningAssignment2-parta", entity='akshaygrao')

wandb: Currently logged in as: akshaygrao (use `wandb login --relogin` to force relogin)


# **BUILD CNN BASIC MODEL**

In [7]:
def cnn_Model(filter_list=[64, 32, 48, 32, 32], dropout=0.2, batch_norm=False, dense_layer_size=512, conv_activations=["relu", "relu", "relu", "relu", "relu"], dense_layer_activation="relu", output_activation="softmax"):

    model = Sequential()

    for i in range(5):
        model.add(Conv2D(filter_list[i], (5, 5), input_shape=(200,200,3), padding="same"))
        if batch_norm:
            model.add(BatchNormalization())
        if(conv_activations[i] == 'leakyrelu'):
          model.add(LeakyReLU())
        elif(conv_activations[i] == 'prelu'):
          model.add(PReLU(alpha_initializer=Constant(value=0.25)))
        else:
          model.add(Activation(conv_activations[i]))
        model.add(MaxPooling2D(pool_size=(2, 2)))
    
    model.add(Flatten())
    model.add(Dense(dense_layer_size))
    model.add(Dropout(dropout))
    if(dense_layer_activation == 'leakyrelu'):
      model.add(LeakyReLU())
    elif(dense_layer_activation == 'prelu'):
      model.add(PReLU(alpha_initializer=Constant(value=0.25)))
    else:
      model.add(Activation(dense_layer_activation))
    model.add(Dense(10))
    model.add(Activation(output_activation))

    return model;

**DATA GENERATORS -PRE PROCESSING OF IMAGE TRAIN ,VAL AND TEST DATA **

In [8]:
def dataset_Generators(train_data_aug=False):
    if train_data_aug:
        train_idg = ImageDataGenerator(
            rescale=1. /255,
            validation_split=0.1,              
            horizontal_flip=True,
            vertical_flip=False,
            rotation_range=15,
            shear_range=0.1,
            zoom_range=0.1,
            width_shift_range=0.2,
            height_shift_range=0.2
        )
    else:
        train_idg = ImageDataGenerator(
            rescale=1. /255,
            validation_split=0.1                 
        )
   #10% of data as validation data is to be kept aside 
    test_idg = ImageDataGenerator(
            rescale=1. /255
        )
    
    print(f"{train_Data_Path}")
    train_Generator = train_idg.flow_from_directory(
        train_Data_Path,
        batch_size=batch_size,
        target_size=(width, height),
        subset='training',
        class_mode='categorical'
    )

    val_Generator = train_idg.flow_from_directory(
        train_Data_Path,
        batch_size=batch_size,
        target_size=(width, height),
        subset='validation',
        class_mode='categorical'
    )

    print(f"{test_Data_Path}")
    test_Generator = test_idg.flow_from_directory(
        test_Data_Path,
        batch_size=batch_size, 
        target_size=(width, height)
    )
    
    return train_Generator, val_Generator, test_Generator

DEFAULT CONFIG FOR SWEEPS

In [9]:
default_config = {
        "filter_list":[64, 32, 48, 32, 32], 
        "dropout":0.2, 
        "batch_norm":False, 
        "dense_layer_size":512,
        "epochs": 10,
        "lr": 0.001,
        "train_data_aug":False,
        "activations": "relu"
    }

#Keras callback    
history = History()

# **CODE FOR HYPERPARAMETER TUNING**

In [10]:
def HP_tuning_run():
    # Create a MirroredStrategy.
    if tf.config.list_physical_devices('GPU'):
        strategy = tf.distribute.MirroredStrategy()
    else:  # use default strategy
        strategy = tf.distribute.get_strategy()
    print('Number of devices: {}'.format(strategy.num_replicas_in_sync))

    # wandb.init(config=default_config, magic=True, project='DLAssignment2', entity='cs21s002-ee21s113-dlassignment-1')
    wandb.init(config=default_config, magic=True, project='DeepLearningAssignment2-parta', entity='akshaygrao')
    config = wandb.config
    print("Config: "+str(config))
    run_name = str(config).replace("{", "").replace("}","").replace(":","-")
    wandb.run.name = run_name

    # Open a strategy scope and create the model
    with strategy.scope():
      acts = [config.activations] * 5
      model = cnn_Model(config.filter_list, config.dropout, config.batch_norm, config.dense_layer_size,dense_layer_activation=config.activations,conv_activations=acts)

    plot_model(model)

    model.summary()

    train_Generator, val_Generator, test_Generator = dataset_Generators(config.train_data_aug)

    model.compile(loss="categorical_crossentropy", optimizer=keras.optimizers.Adam(config.lr), metrics="categorical_accuracy")

    model.fit(train_Generator, epochs=config.epochs, validation_data=val_Generator, callbacks=[WandbCallback(), history])

    model.evaluate(test_Generator)

    model.save(f'{run_name.replace(",","-")}.h5')  # creates a HDF5 file 'my_model.h5'

**SWEEP CONFIG FOR HP TUNING USING WAND-b**

In [11]:
sweep_config = {
    "name": "Assignment 2 - alpha",
    "method": "bayes",
    "metric":{
      "goal": "maximize",
      "name": "val_categorical_accuracy"
    },
    "early_terminate":{
      "type": "hyperband",
      "min_iter": 2,
      "eta":2
    },
    "project": 'Assignment 2',
    "parameters": {
        "filter_list": {
            "values": [[64,128,350,256,128]]
        },
        "dropout": {
            "values": [0.5,0.7,0.6]
        },
        "batch_norm": {
            "values": [True,False]
        },
        "dense_layer_size": {
            "values": [5096]
        },
        "train_data_aug": {
            "values": [True,False]
        },
        "epochs": {
            "values": [8]
        },
        "lr": {
            "values": [0.0001]
        },
        "activations": {
            "values":[ 'leakyrelu','prelu']
        }
        
    }
}

In [12]:
# sweep_id = wandb.sweep(sweep_config, project='DLAssignment2', entity='cs21s002-ee21s113-dlassignment-1')
# sweep_id = wandb.sweep(sweep_config,  project='DeepLearningAssignment2-parta', entity='akshaygrao')

In [13]:
# wandb.agent(sweep_id, function=HP_tuning_run, project='DLAssignment2', entity='cs21s002-ee21s113-dlassignment-1')
# wandb.agent(sweep_id, function=HP_tuning_run, project='DeepLearningAssignment2-parta', entity='akshaygrao')

**RETRIEVE THE BEST MODEL AND RUN THE MODEL FOR PREDICTIONS**

In [14]:
def run_best_model():
  #  MirroredStrategy in GPU
  if tf.config.list_physical_devices('GPU'):
      strategy = tf.distribute.MirroredStrategy()
  else: 
      strategy = tf.distribute.get_strategy()
  print('Number of devices: {}'.format(strategy.num_replicas_in_sync))


  with strategy.scope():
      model = cnn_Model(filter_list=[64,128,256,256,128], dropout = 0.7, batch_norm = False, dense_layer_size= 4096,conv_activations=['leakyrelu','leakyrelu','leakyrelu','leakyrelu','leakyrelu'],dense_layer_activation='leakyrelu')

  plot_model(model)

  model.summary()

  train_Generator, val_Generator, test_Generator = dataset_Generators(False)

  model.compile(loss="categorical_crossentropy", optimizer=keras.optimizers.Adam(0.0001), metrics="accuracy")

  model.fit(train_Generator, epochs=8, validation_data=val_Generator, callbacks=[history])

  model.evaluate(test_Generator)
  return model,test_Generator

POPULATING THE PREDICTION IN A TABLE/GRID

In [15]:
def create_grid_from_best_model(model,test_Generator,pred_classes,images,labels,predictions):
  
  count = [0]*10
  images_list = []
  f, a = plt.subplots(nrows=10, ncols=3, figsize=(15,50))
  columns=["id", "image", "Predictions"]
  wandb.Table.MAX_ROWS = 20
  test_table = wandb.Table(columns=columns)
  for count, img in enumerate(images):
      class_label = np.argmax(labels[count], axis=0)
      pred_label = np.argmax(predictions[count], axis=0)
      if count[class_label] < 3:
          a[class_label, count[class_label]].imshow(img)
          a[class_label, count[class_label]].axis('off')
          ttl = "Actual: " + pred_classes[class_label] + "\nPredicted: " + pred_classes[pred_label]
          a[class_label, count[class_label]].set_title(ttl)
          count[class_label] += 1
          images_list.append((img, ttl))
          test_table.add_data(count,wandb.Image(img),ttl)
          wandb.log({"Prediction Grid Table":test_table}) 

          
  plt.show()

**VISUALIZING THE 1ST INITIAL FILTER OUTPUT OF BEST MODEL**

In [16]:
def visualize_filter_of_best_model(model):
  layer_outputs = [layer.output for layer in model.layers]

  activation_model = Model(inputs=model.input, outputs=layer_outputs) # Creates a model that will return these outputs, given the model input
  activations = activation_model.predict(images[55:56])
  first_layer_activation = activations[0]
  print(first_layer_activation.shape)
  plt.matshow(first_layer_activation[0, :, :, 63])

  plt.figure(figsize=(20,40))
  for d in range(64):
    plt.subplot(16, 8, d+1)
    plt.imshow(first_layer_activation[0, :, :, d])
    wandb.log({"Filters": wandb.Image(first_layer_activation[0, :, :, d])})
    plt.savefig("filters.png")
  plt.show()

  filters_img = Image.open("filters.png") 
  wandb.log({"Filters of Layer 1": wandb.Image(filters_img)})

# **CODE FOR GUIDED BACK PROPOGATION **

In [17]:
def generate_guided_propogation_plots():
  model_upto_conv5 = tf.keras.models.Model([model.inputs],[model.get_layer("conv2d_4").output]) 
  @tf.custom_gradient
  def guidedRelU(x):
      def grad(dy):
          return tf.cast(dy>0, tf.float32) * tf.cast(x>0, tf.float32) * dy
      return tf.nn.relu(x), grad

  for layer in model.layers[1:]:
      if hasattr(layer, 'activation') and layer.activation==tf.keras.activations.relu:
          layer.activation = guidedRelU

  f, a = plt.subplots(10, 2, figsize=(10, 40))
  inputs = tf.convert_to_tensor(images[55:65], dtype=tf.float32)

  for i in range(10):

      with tf.GradientTape() as tape:
          input = tf.expand_dims(inputs[i], 0)
          tape.watch(input)
          out = model_upto_conv5(input)[0]
      
      gradients = tape.gradient(out,input)[0]

      a[i][0].set_title("Original")
      a[i][0].imshow(inputs[i])
      a[i][1].set_title("Guided backpropagation")
      a[i][1].imshow(gradients)
  wandb.log({"Guided Backpropagation": wandb.Image(f)})
  f.savefig("guided_backprop.png")
  plt.show()


Run custom model

In [82]:
def run_model(model):

  plot_model(model)

  model.summary()

  train_Generator, val_Generator, test_Generator = dataset_Generators(False)

  model.compile(loss="categorical_crossentropy", optimizer=keras.optimizers.Adam(0.0001), metrics="accuracy")

  model.fit(train_Generator, epochs=8, validation_data=val_Generator, callbacks=[history])

  model.evaluate(test_Generator)

### Alternative for command line (uncomment and run for plots, filter visualization)

In [ ]:
model,test_Generator = run_best_model()
pred_classes = {0: 'Amphibia', 1: 'Animalia', 2: 'Arachnida', 3: 'Aves', 4: 'Fungi', 5: 'Insecta', 6: 'Mammalia', 7: 'Mollusca', 8: 'Plantae', 9: 'Reptilia'}

images, labels = test_Generator.next()
predictions = model(images)
create_grid_from_best_model(model,test_Generator,pred_classes,images,labels,predictions)
visualize_filter_of_best_model(model)

Error in callback <bound method _WandbInit._resume_backend of <wandb.sdk.wandb_init._WandbInit object at 0x7fdd7c21e7d0>> (for pre_run_cell):


Exception: ignored

Number of devices: 1
Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_10 (Conv2D)          (None, 200, 200, 64)      4864      
                                                                 
 leaky_re_lu_6 (LeakyReLU)   (None, 200, 200, 64)      0         
                                                                 
 max_pooling2d_10 (MaxPoolin  (None, 100, 100, 64)     0         
 g2D)                                                            
                                                                 
 conv2d_11 (Conv2D)          (None, 100, 100, 128)     204928    
                                                                 
 leaky_re_lu_7 (LeakyReLU)   (None, 100, 100, 128)     0         
                                                                 
 max_pooling2d_11 (MaxPoolin  (None, 50, 50, 128)      0         
 g2D)                            

##**Command line argument handler**

In [83]:
def init_argparse() -> argparse.ArgumentParser:
    parser = argparse.ArgumentParser()

    parser.add_argument("-pgb","--plot_guided_bp", action="store_true",dest='plotbackprop', help="Plot guided backprop on last model(call only after running a model)")
    parser.add_argument("-visfilters", "--visualize_filters", action="store_true",dest='visfiltersbestmodel', help="Visualize filters of last model(call only after running a model)")
    parser.add_argument("-cgbm", "--create_grid_model", action="store_true",dest='creategridbstmodel', help="Create grid for last model(call only after running a model)")
    parser.add_argument("-runbest", "--run_best_model", action="store_true",dest='runbestmodel', help="Run best model")
    
    subparsers = parser.add_subparsers(dest='command')
    run_model_parser = subparsers.add_parser('run_model')
    run_model_parser.add_argument("--dropout", action="store",dest='dropout', help="Specify dropout param",default=0.2)
    run_model_parser.add_argument("--batch_norm", action="store",dest='batch_norm', help="Specify batch normalization param",default=False)
    run_model_parser.add_argument("--dense_layer_size", action="store",dest='dense_layer_size', help="Specify number of neurons in dense layer param",default=512)
    run_model_parser.add_argument("--conv_activations_conv", action="append",dest='conv_activations_conv', help="Specify activation in convolution layer param")
    run_model_parser.add_argument("--conv_activations_out", action="store",dest='conv_activations_out', help="Specify activation in output layer param",default='softmax')
    run_model_parser.add_argument("--conv_activations_dense", action="store",dest='conv_activations_dense', help="Specify activation in dense layer param",default='relu')
    run_model_parser.add_argument("--filter_size_list", action="append",dest='filter_size_list', help="Specify number of filters in each layer")

    return parser

In [85]:
if __name__ == "__main__":
  parser = init_argparse()
  args = parser.parse_args(['--help'])

  model = None
  test_Generator = None
  
  if(args.command == 'run_model'):
    conv_activations_conv = args.conv_activations_conv 
    filter_size_list = args.filter_size_list
    if(conv_activations_conv is None):
      conv_activations_conv = ["relu", "relu", "relu", "relu", "relu"]
    if(filter_size_list is None):
      filter_size_list = [64, 32, 48, 32, 32]
    
    model = cnn_Model(filter_list=filter_size_list, dropout=args.dropout, batch_norm=args.batch_norm, dense_layer_size=args.dense_layer_size, conv_activations=args.conv_activations_conv, dense_layer_activation=args.conv_activations_dense, output_activation=args.conv_activations_out)
    model,test_Generator = run_model(model)

  if(args.runbestmodel):
    print("Running best model----")
    model,test_Generator = run_best_model()
  if(args.creategridbstmodel):
    print("Creating grid with best model")
    pred_classes = {0: 'Amphibia', 1: 'Animalia', 2: 'Arachnida', 3: 'Aves', 4: 'Fungi', 5: 'Insecta', 6: 'Mammalia', 7: 'Mollusca', 8: 'Plantae', 9: 'Reptilia'}

    images, labels = test_Generator.next()
    predictions = model(images)
    create_grid_from_best_model(model,test_Generator,pred_classes,images,labels,predictions)
  if(args.plotbackprop):
    print("Plotting guided backpropogation")
    generate_guided_propogation_plots()
  if(args.visfiltersbestmodel):
    print("Visualizing filters of best model")
    visualize_filter_of_best_model(model)


usage: ipykernel_launcher.py [-h] [-pgb] [-visfilters] [-cgbm] [-runbest]
                             {run_model} ...

positional arguments:
  {run_model}

optional arguments:
  -h, --help            show this help message and exit
  -pgb, --plot_guided_bp
                        Plot guided backprop on last model(call only after
                        running a model)
  -visfilters, --visualize_filters
                        Visualize filters of last model(call only after
                        running a model)
  -cgbm, --create_grid_model
                        Create grid for last model(call only after running a
                        model)
  -runbest, --run_best_model
                        Run best model


SystemExit: ignored

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2890: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
